### Advent of Code: Day 1, Part 2

For the second part, we're using the [same data](./day_2_input.txt)! 

We are adding the frequency day over day and identifying the first value that is repeated.

#### My process: 
I usually tackle problems by finding a solution that *works*, then improving that solution.

A solution that *would* work is to just create a second list that tracks the resulting frequency. Once the list is built, we can grab the first value that is duplicated. Let's try that.

In [33]:
# Open file.
file = open('./day_1_input.txt', 'r')

# Save day_2_input as list called current_freq.
current_freq = [int(i) for i in file]

# Create list of resulting frequencies called resulting_freq.
resulting_freq = [sum(current_freq[:i+1]) for i in range(len(current_freq))]

I initially made a mistake and just wrote `[i for i in file]` for current frequency, causing an integer/string issue.

Let's make sure that this happened the correct way.

In [2]:
current_freq[0:10]

[-10, 18, 5, 1, 1, -19, -13, -4, -4, -5]

In [3]:
resulting_freq[0:10]

[-10, 8, 13, 14, 15, -4, -17, -21, -25, -30]

In looking at the first ten values in `resulting_freq`, it does seem to be the resulting frequency after each step.

Now, we need to find the first duplicated value in `resulting_freq` and return that. Since sets do not allow duplicated objects, I'll plan to create a set of items 0 through `i` in `resulting_freq`. When the items are all unique, the length of that set should be `i`. If there's a duplicate in it, then that length shouldn't be `i`.

In [6]:
len(set(resulting_freq[0:10])) # All unique values.

10

In [10]:
len(set(resulting_freq[0:10000])) # Some duplicates exist!

1016

So I run this, thinking it'll work:

In [36]:
for i in range(1, len(resulting_freq)):
    if len(set(resulting_freq[0:i])) != i:
        print(i, resulting_freq[i])

No output. I play around with it, trying to figure out what I did wrong. I try reloading `resulting_freq` to make sure that I didn't overwrite it the wrong way, I try printing out the different elements in the set of `resulting_freq`. Is it possible that there aren't duplicates?

In [34]:
len(resulting_freq)

1016

In [35]:
len(set(resulting_freq))

1016

Ahhh. So it's printing nothing out because it shouldn't be printing something out. There aren't any duplicates. I [played myself earlier](https://www.google.com/url?sa=i&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwiX1vbW-o3fAhUJZd8KHQG6D3kQjRx6BAgBEAU&url=https%3A%2F%2Fknowyourmeme.com%2Fphotos%2F1228324-congratulations-you-played-yourself&psig=AOvVaw3MsfQvlDLv4DJSwbsJyTkl&ust=1544280332491680) when I wrote 
> `len(set(resulting_freq[0:10000])) # Some duplicates exist!`

I just indexed my list with an index that didn't actually exist, and the result of `1016` fooled me into thinking there were lots of duplicates.

I checked the prompt again: "Note that your device might need to repeat its list of frequency changes many times before a duplicate frequency is found, and that duplicates might be found while in the middle of processing the list."

My new strategy is to create a `while` loop to have `current_freq` used over and over again, then calculate `resulting_freq` and find the answer.

In [ ]:
current_freq = [int(i) for i in file]
resulting_freq = [sum(current_freq[:i+1]) for i in range(len(current_freq))]

i = 1
while True:
    if len(set(resulting_freq[0:i])) != i:
        print(i, resulting_freq[i])
        break
    i += 1
#    resulting_freq.append(sum)
    mod = i % len(current_freq)

I'm abandoning this approach because, mid-coding, I realized that we can just create `resulting_freq` in a much easier way...

In [39]:
resulting_freq = [sum(current_freq[:2])]
i = 1
while True:
    mod = i % len(current_freq)
    resulting_freq.append(resulting_freq[-1] + current_freq[mod])
    if len(set(resulting_freq[0:i])) != i:
        print(i, resulting_freq[i])
        break
    i += 1

KeyboardInterrupt: 

This ran for awhile and didn't return anything, so I paused it. Did things work the way I expected?

In [41]:
i, resulting_freq[i]

(74287, 37903)

Seems right. Let's add a timer and an output every 10,000 iterations to let me know it's still working.

In [48]:
import time

t0 = time.time()
resulting_freq = [sum(current_freq[:2])]
i = 2
while True:
    if i % 10000 == 0:
        print(f"We have {i} iterations in {round(time.time() - t0, 1)} seconds.")
    mod = i % len(current_freq)
    resulting_freq.append(resulting_freq[-1] + current_freq[mod])
    if len(set(resulting_freq)) != i:
        print(i-1, resulting_freq[-1])
        break
    i += 1

We have 10000 iterations in 1.6 seconds.
We have 20000 iterations in 5.1 seconds.
We have 30000 iterations in 12.7 seconds.
We have 40000 iterations in 21.2 seconds.
We have 50000 iterations in 31.4 seconds.
We have 60000 iterations in 42.6 seconds.
We have 70000 iterations in 54.7 seconds.
We have 80000 iterations in 71.5 seconds.
We have 90000 iterations in 99.4 seconds.
We have 100000 iterations in 128.3 seconds.
We have 110000 iterations in 158.4 seconds.
We have 120000 iterations in 186.7 seconds.
We have 130000 iterations in 214.0 seconds.
We have 140000 iterations in 242.0 seconds.
145799 287


I got the wrong answer three times before finding the correct one:
- I initially guessed 145,801, but then realized they wanted the resulting frequency rather than the number of iterations.
- I then guessed 317, which was too high. 
- I checked my indices, made some tweaks, and got 305. Still too high.
- I re-ran it and got 287, which was correct!

In [45]:
i-1

145800

In [46]:
resulting_freq[i-1]

305

#### How might I improve this?


Well, searching from 1 to a potentially very high number is tough. Rather than incrementing `i` by 1, my gut tells me that I could probably square `i` each time to more effectively search.

However, I should first see if it's quicker to use a list comprehension and just directly fill the list out versus appending to my list time over time. 

In [54]:
t0 = time.time()

i = 1000000
lst = [sum(current_freq[:j+1]) for j in range(i)]

print(time.time() - t0)

14.62417197227478


In [55]:
t0 = time.time()

lst = [current_freq[0]]

for i in range(1000000):
    mod = i % len(current_freq)
    lst.append(lst[-1] + current_freq[mod])

print(time.time() - t0)

0.7400457859039307


In [69]:
len(current_freq)

1016

Okay. It's considerably faster (in this case) to append than to use a list comprehension. Let's do that, then.

In [114]:
t0 = time.time()

resulting_freq = [sum(current_freq[:1])]
last_i = 1
i = 10

while True:
    for j in range(last_i, i):
        mod = j % len(current_freq)
        resulting_freq.append(resulting_freq[-1] + current_freq[mod])
    if len(set(resulting_freq)) != i:
        break
    print(f"We have {i} iterations in {round(time.time() - t0, 1)} seconds.")
    last_i = i
    i *= 2

print(f"Changing loops - answer is between {last_i} and {i}. Time elapsed is {round(time.time() - t0, 1)} seconds.")
for k in range(last_i, i+1):
    if k % 10000 == 0:
        print(f"We're on iteration {k} (from {last_i} to {i}) in {round(time.time() - t0, 1)} seconds.")
    if len(set(resulting_freq[0:k+1])) != k+1:
        print(k-1, resulting_freq[k-1])
        break

We have 10 iterations in 0.0 seconds.
We have 20 iterations in 0.0 seconds.
We have 40 iterations in 0.0 seconds.
We have 80 iterations in 0.0 seconds.
We have 160 iterations in 0.0 seconds.
We have 320 iterations in 0.0 seconds.
We have 640 iterations in 0.0 seconds.
We have 1280 iterations in 0.0 seconds.
We have 2560 iterations in 0.0 seconds.
We have 5120 iterations in 0.0 seconds.
We have 10240 iterations in 0.0 seconds.
We have 20480 iterations in 0.0 seconds.
We have 40960 iterations in 0.0 seconds.
We have 81920 iterations in 0.1 seconds.
Changing loops - answer is between 81920 and 163840. Time elapsed is 0.1 seconds.
We're on iteration 90000 (from 81920 to 163840) in 26.9 seconds.
We're on iteration 100000 (from 81920 to 163840) in 59.9 seconds.
We're on iteration 110000 (from 81920 to 163840) in 95.1 seconds.
We're on iteration 120000 (from 81920 to 163840) in 134.1 seconds.
We're on iteration 130000 (from 81920 to 163840) in 173.3 seconds.
We're on iteration 140000 (from 81

This took a lot of debugging - indices throw *everything* off - but I was able to get an answer that took about... the same amount of time. The initial search went by fast, but after changing loops we used way more time.

Now what I want to do is implement [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method) to reduce the amount of time in that second loop. In short: rather than searching each value from 81920 to 163840, I'll try to narrow down the search by half each time by checking the value in the middle and eliminating half of it.

In [122]:
t0 = time.time()

resulting_freq = [sum(current_freq[:1])]
last_i = 1
i = 10

while True:
    for j in range(last_i, i):
        mod = j % len(current_freq)
        resulting_freq.append(resulting_freq[-1] + current_freq[mod])
    if len(set(resulting_freq)) != i:
        break
    print(f"We have {i} iterations in {round(time.time() - t0, 1)} seconds.")
    last_i = i
    i *= 2

print(f"Changing loops - answer is between {last_i} and {i}. Time elapsed is {round(time.time() - t0, 1)} seconds.")
# for k in range(last_i, i+1):
#     if k % 10000 == 0:
#         print(f"We're on iteration {k} (from {last_i} to {i}) in {round(time.time() - t0, 1)} seconds.")    

min_value = last_i
max_value = i

while True:
    mean_value = round((min_value + max_value) / 2)
    print(f"Min: {min_value}, Mean: {mean_value}, Max: {max_value}")
    
    if len(set(resulting_freq[0:(mean_value + 1)])) != (mean_value + 1):
        if max_value - min_value <= 1:
            print(mean_value+1, resulting_freq[mean_value+1])
            break
        else:
            max_value = mean_value
    else:
        min_value = mean_value

print(f"This all took {time.time() - t0} seconds!")

We have 10 iterations in 0.0 seconds.
We have 20 iterations in 0.0 seconds.
We have 40 iterations in 0.0 seconds.
We have 80 iterations in 0.0 seconds.
We have 160 iterations in 0.0 seconds.
We have 320 iterations in 0.0 seconds.
We have 640 iterations in 0.0 seconds.
We have 1280 iterations in 0.0 seconds.
We have 2560 iterations in 0.0 seconds.
We have 5120 iterations in 0.0 seconds.
We have 10240 iterations in 0.0 seconds.
We have 20480 iterations in 0.0 seconds.
We have 40960 iterations in 0.0 seconds.
We have 81920 iterations in 0.1 seconds.
Changing loops - answer is between 81920 and 163840. Time elapsed is 0.1 seconds.
Min: 81920, Mean: 122880, Max: 163840
Min: 122880, Mean: 143360, Max: 163840
Min: 143360, Mean: 153600, Max: 163840
Min: 143360, Mean: 148480, Max: 153600
Min: 143360, Mean: 145920, Max: 148480
Min: 143360, Mean: 144640, Max: 145920
Min: 144640, Mean: 145280, Max: 145920
Min: 145280, Mean: 145600, Max: 145920
Min: 145600, Mean: 145760, Max: 145920
Min: 145760, Me

By implementing Newton's method, I took the amount of time this took from around 270 seconds down to under one-fifth of a second.

### Resources Used
- https://realpython.com/python-f-strings/#f-strings-a-new-and-improved-way-to-format-strings-in-python